In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
import re
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [2]:
def get_url(category, tot_pages):
    for page in range(1,tot_pages+1):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')       
        
        url_main = f"https://mir-sporta.com/category/{category}/?page={page}"
        response = requests.get(url_main, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = 'c-product-thumb__name c-link c-link_style_hover')
        for item in data:
            item_url = 'https://mir-sporta.com' + item.attrs['href']
            yield item_url

In [3]:
def get_item(category, tot_pages):
    
    for url_item in get_url(category, tot_pages):
        response = requests.get(url_item, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        
        try:
            title = soup.find('h1').text
        except:
            title = ''

        try:
            article = oup.find(class_ = 'c-value').text
            article = article.replace(' ', '').replace('\n', '')
            article = article.replace('Артикул:', '')
        except:
            article = ''

        try:
            image = soup.find(class_ = 'l-image-box l-image-box_fill', href = True)
            url = image['href']

        except:
            print(f'Log: no image for {article} found...')
            url = ''
   
        try:
            desc = soup.find('h2').next_sibling.text
            desc = desc.replace('\n', ' ')
        except:
            desc = ''

        try:
            char = soup.find(class_ = 'c-product-features c-product-features_2_cols').text
            char = re.sub(r"(?<=\w)([А-ЯA-Z])", r" \1", char)
            char = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", char).strip()
            char = char.replace('Основные', '').replace('Примечание Технические характеристики, описание, комплектация поставки и внешний вид товаров не является публичной офертой. Подробную информацию уточняйте у наших менеджеров.', '')
        except:
            char = ''
        
        try:
            price_lst = []
            price_info = soup.find(class_ = 'c-product-add-to-cart__prices')
            for price in price_info:
                price = price.text.replace('₽', '').replace(' ', '')
                price_lst.append(price)
            price_fin = price_lst[0]


        except:
            price = ''

        try:
            cat = soup.find(class_ = 'c-breadcrumbs c-breadcrumbs_current-item-mode_show').text
            temp_str = re.sub(re.compile('[^а-яА-Я ]'), '', cat)
            lst = re.findall('[А-Я][^А-Я]*', temp_str)

            try:
                cat_1 = lst[-3]
            except:
                cat_1 = ''
            try:
                cat_2 = lst[-2]
            except:
                cat_2 = ''
            try:
                cat_3 = re.sub(re.compile('[^a-zA-Z ]'), '', cat).strip().split(' ')[0]
            except:
                cat_3 = ''
        except:
            cat_1 = ''
            cat_2 = ''
            cat_3 = ''
    
        yield title, article, price_fin, cat_1, cat_2, cat_3, url, desc, char

In [4]:
df_columns = ['title', 'article','price', 'cat_1', 'cat_2', 'cat_3','url', 
              'description', 'caracteristics']

df = pd.DataFrame(columns = df_columns)
save_frequency = 200
log_frequency = 8

In [5]:
def to_csv(category, tot_pages):
    counter = 0  

    for item in get_item(category, tot_pages):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
            #time.sleep(0.05)

            counter += 1
            if counter % save_frequency == 0:
                df.to_csv('mir-sporta.csv')
                print(f'{counter} items saved in csv..')
    df.to_csv('mir-sporta.csv')

    print(f'\nTotal items saved: {counter}')  

In [6]:
to_csv('begovye-dorozhki', 36)

Log: no image for  found...
Log: working over page 8...
200 items saved in csv..
Log: working over page 16...
400 items saved in csv..
Log: no image for  found...
Log: working over page 24...
600 items saved in csv..
Log: working over page 32...

Total items saved: 713


In [8]:
to_csv('steppery', 6)


Total items saved: 114


In [9]:
df

,title,article,price,cat_1,cat_2,cat_3,url,description,caracteristics
0,Беговая дорожка Koenigsmann T1.0,,32290,Кардиотренажеры,Беговые дорожки,Koenigsmann,https://cdn.mir-sporta.com/wa-data/public/shop...,Беговая дорожка Koenigsmann T1 - это электриче...,Тип дорожки Электрическая Класс тренажера Дом...
1,Беговая дорожка Orlauf OWL grey,,59900,Кардиотренажеры,Беговые дорожки,Orlauf,https://cdn.mir-sporta.com/wa-data/public/shop...,Беговая дорожка Orlauf OWL grey - идеальное ре...,Тип дорожки Электрическая Класс тренажера Дом...
2,Беговая дорожка Koenigsmann B3,,36790,Кардиотренажеры,Беговые дорожки,Koenigsmann,https://cdn.mir-sporta.com/wa-data/public/shop...,Koenigsmann B3 — Первая электрическая беговая ...,Тип дорожки Электрическая Класс тренажера Дом...
3,Беговая дорожка UNIXFIT ST-530M,,34192,Кардиотренажеры,Беговые дорожки,UnixFit,https://cdn.mir-sporta.com/wa-data/public/shop...,Беговая дорожка UNIXFIT ST-530M - это электрич...,Тип дорожки Электрическая Класс тренажера Дом...
4,Беговая дорожка Koenigsmann B2.0,,42990,Кардиотренажеры,Беговые дорожки,Koenigsmann,https://cdn.mir-sporta.com/wa-data/public/shop...,Беговая дорожка Koenigsmann B2 - это электриче...,Тип дорожки Электрическая Класс тренажера Дом...
...,...,...,...,...,...,...,...,...,...
822,Степпер MATRIX Endurance S-ES-F/PLED-C,,721050,Кардиотренажеры,Степперы,MATRIX,https://cdn.mir-sporta.com/wa-data/public/shop...,Степпер MATRIX Endurance S-ES-F/PLED-C - это к...,Тип степпера Классический Опции для рук Непод...
823,Степпер MATRIX Endurance S-ES-F/LED-C,,597550,Кардиотренажеры,Степперы,MATRIX,https://cdn.mir-sporta.com/wa-data/public/shop...,Степпер MATRIX Endurance S-ES-F/LED-C - это ид...,Тип степпера Классический Опции для рук Непод...
824,Степпер Insight PP580,,323510,Кардиотренажеры,Степперы,Insight,https://cdn.mir-sporta.com/wa-data/public/shop...,Степпер Insight PP580 - это инновационный трен...,Система нагружения Электромагнитная Уровни на...
825,Степпер Atlas Sport AS-504 зеленый,,6200,Кардиотренажеры,Степперы,Atlas,https://cdn.mir-sporta.com/wa-data/public/shop...,Степпер Atlas Sport AS-504 зеленый - это класс...,Тип степпера Классический Система нагружения ...
